In [1]:
import numpy as np
import unittest

In [2]:
# Fuzzy TOPSIS implementation
def fuzzy_topsis(decision_matrix, weights, criteria_type):
    """
    Implementation of the Fuzzy TOPSIS method for multicriteria decision analysis.

    Parameters:
    decision_matrix (ndarray): Matrix (m x n x 3) with m alternatives, n criteria, and 3 values representing fuzzy numbers (low, medium, high).
    weights (list or ndarray): Weights for each criterion.
    criteria_type (list): A list specifying whether each criterion should be 'max' or 'min'.

    Returns:
    ndarray: Score for each alternative, ranking them from best to worst.
    """
    # Step 1: Normalize the decision matrix
    norm_matrix = np.zeros_like(decision_matrix, dtype=float)
    for j in range(decision_matrix.shape[1]):
        if criteria_type[j] == 'max':
            max_value = np.max(decision_matrix[:, j, 2])
            norm_matrix[:, j, 0] = decision_matrix[:, j, 0] / max_value
            norm_matrix[:, j, 1] = decision_matrix[:, j, 1] / max_value
            norm_matrix[:, j, 2] = decision_matrix[:, j, 2] / max_value
        else:
            min_value = np.min(decision_matrix[:, j, 0])
            norm_matrix[:, j, 0] = min_value / decision_matrix[:, j, 2]
            norm_matrix[:, j, 1] = min_value / decision_matrix[:, j, 1]
            norm_matrix[:, j, 2] = min_value / decision_matrix[:, j, 0]

    # Step 2: Apply the weights
    weighted_matrix = np.zeros_like(norm_matrix, dtype=float)
    for j in range(norm_matrix.shape[1]):
        weighted_matrix[:, j, 0] = norm_matrix[:, j, 0] * weights[j]
        weighted_matrix[:, j, 1] = norm_matrix[:, j, 1] * weights[j]
        weighted_matrix[:, j, 2] = norm_matrix[:, j, 2] * weights[j]

    # Step 3: Determine FPIS and FNIS
    fpis = np.zeros((decision_matrix.shape[1], 3), dtype=float)
    fnis = np.zeros((decision_matrix.shape[1], 3), dtype=float)
    for j in range(weighted_matrix.shape[1]):
        if criteria_type[j] == 'max':
            fpis[j] = np.max(weighted_matrix[:, j, :], axis=0)
            fnis[j] = np.min(weighted_matrix[:, j, :], axis=0)
        else:
            fpis[j] = np.min(weighted_matrix[:, j, :], axis=0)
            fnis[j] = np.max(weighted_matrix[:, j, :], axis=0)

    # Step 4: Calculate the distances to FPIS and FNIS
    distance_to_fpis = np.sqrt(np.sum(np.power(weighted_matrix - fpis, 2), axis=(1, 2)))
    distance_to_fnis = np.sqrt(np.sum(np.power(weighted_matrix - fnis, 2), axis=(1, 2)))

    # Step 5: Calculate the similarity to the ideal solution
    with np.errstate(divide='ignore', invalid='ignore'):
        similarity_to_ideal = distance_to_fnis / (distance_to_fpis + distance_to_fnis)
        similarity_to_ideal[np.isnan(similarity_to_ideal)] = 0  # Handle division by zero

    return similarity_to_ideal


In [3]:
# Unit tests for Fuzzy TOPSIS with formatted output
class TestFuzzyTOPSIS(unittest.TestCase):
    def test_case_1(self):
        decision_matrix_fuzzy = np.array([
            [[2, 3, 4], [3, 4, 5], [1, 2, 3], [4, 5, 6]],
            [[1, 2, 3], [2, 3, 4], [2, 3, 4], [3, 4, 5]],
            [[3, 4, 5], [4, 5, 6], [3, 4, 5], [2, 3, 4]],
            [[4, 5, 6], [5, 6, 7], [4, 5, 6], [1, 2, 3]],
            [[2, 3, 4], [3, 4, 5], [1, 2, 3], [5, 6, 7]]
        ])
        weights = [0.4, 0.3, 0.2, 0.1]
        criteria_type = ['max', 'max', 'min', 'max']

        score = fuzzy_topsis(decision_matrix_fuzzy, weights, criteria_type)
        ranking = np.argsort(score)[::-1]

        # Formatted results display for the test case
        print("\nTest case 1 - Results:")
        print("| Alternative | Score   | Rank  |")
        print("|-------------|---------|-------|")
        for i, alternative in enumerate(ranking):
            print(f"| Alternative {alternative + 1:<2} | {score[alternative]:<7.4f} | {i + 1:<5} |")

        # Corrected assertion
        expected_top_alternative = 4  # Alternative number (1-based indexing)
        self.assertEqual(ranking[0] + 1, expected_top_alternative,
                         f"Test case 1 failed: Expected top alternative to be {expected_top_alternative}")

    def test_case_2(self):
        decision_matrix_fuzzy = np.array([
            [[1, 2, 3], [4, 5, 6], [2, 3, 4], [5, 6, 7]],
            [[2, 3, 4], [3, 4, 5], [3, 4, 5], [4, 5, 6]],
            [[3, 4, 5], [2, 3, 4], [4, 5, 6], [3, 4, 5]],
            [[4, 5, 6], [1, 2, 3], [5, 6, 7], [2, 3, 4]],
            [[5, 6, 7], [1, 2, 3], [1, 2, 3], [1, 2, 3]]
        ])
        weights = [0.3, 0.3, 0.2, 0.2]
        criteria_type = ['max', 'max', 'min', 'max']

        score = fuzzy_topsis(decision_matrix_fuzzy, weights, criteria_type)
        ranking = np.argsort(score)[::-1]

        # Formatted results display for the test case
        print("\nTest case 2 - Results:")
        print("| Alternative | Score   | Rank  |")
        print("|-------------|---------|-------|")
        for i, alternative in enumerate(ranking):
            print(f"| Alternative {alternative + 1:<2} | {score[alternative]:<7.4f} | {i + 1:<5} |")

        # Corrected assertion
        expected_top_alternative = 2  # Alternative number (1-based indexing)
        self.assertEqual(ranking[0] + 1, expected_top_alternative,
                         f"Test case 2 failed: Expected top alternative to be {expected_top_alternative}")

    def test_case_3(self):
        decision_matrix_fuzzy = np.array([
            [[5, 6, 7], [3, 4, 5], [1, 2, 3], [2, 3, 4]],
            [[4, 5, 6], [4, 5, 6], [2, 3, 4], [3, 4, 5]],
            [[3, 4, 5], [5, 6, 7], [3, 4, 5], [4, 5, 6]],
            [[2, 3, 4], [6, 7, 8], [4, 5, 6], [5, 6, 7]],
            [[1, 2, 3], [7, 8, 9], [5, 6, 7], [6, 7, 8]]
        ])
        weights = [0.2, 0.4, 0.3, 0.1]
        criteria_type = ['max', 'max', 'min', 'max']

        score = fuzzy_topsis(decision_matrix_fuzzy, weights, criteria_type)
        ranking = np.argsort(score)[::-1]

        # Formatted results display for the test case
        print("\nTest case 3 - Results:")
        print("| Alternative | Score   | Rank  |")
        print("|-------------|---------|-------|")
        for i, alternative in enumerate(ranking):
            print(f"| Alternative {alternative + 1:<2} | {score[alternative]:<7.4f} | {i + 1:<5} |")

        # Corrected assertion
        expected_top_alternative = 5  # Alternative number (1-based indexing)
        self.assertEqual(ranking[0] + 1, expected_top_alternative,
                         f"Test case 3 failed: Expected top alternative to be {expected_top_alternative}")


In [4]:
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK



Test case 1 - Results:
| Alternative | Score   | Rank  |
|-------------|---------|-------|
| Alternative 4  | 0.8176  | 1     |
| Alternative 3  | 0.6642  | 2     |
| Alternative 5  | 0.3457  | 3     |
| Alternative 1  | 0.3270  | 4     |
| Alternative 2  | 0.2192  | 5     |

Test case 2 - Results:
| Alternative | Score   | Rank  |
|-------------|---------|-------|
| Alternative 2  | 0.5322  | 1     |
| Alternative 1  | 0.5294  | 2     |
| Alternative 3  | 0.5088  | 3     |
| Alternative 4  | 0.4839  | 4     |
| Alternative 5  | 0.4441  | 5     |

Test case 3 - Results:
| Alternative | Score   | Rank  |
|-------------|---------|-------|
| Alternative 5  | 0.6777  | 1     |
| Alternative 4  | 0.6723  | 2     |
| Alternative 3  | 0.5962  | 3     |
| Alternative 2  | 0.4649  | 4     |
| Alternative 1  | 0.3223  | 5     |
